In [1]:
import pandas as pd
import numpy as np
import datetime
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns
import talib as ta

In [2]:
#Read data
data_xau = pd.read_csv('data/analysis/D1_XAUUSD_Returns.csv', parse_dates=[0], index_col='Date')
data_xag = pd.read_csv('data/analysis/D1_XAGUSD_Returns.csv', parse_dates=[0], index_col='Date')
data_eur = pd.read_csv('data/analysis/D1_EURUSD_Returns.csv', parse_dates=[0], index_col='Date')


In [3]:
print(data_xau.head())
print(data_xag.head())
print(data_eur.head())

              Open    High     Low   Close  Volume    Weekday  Yearday  \
Date                                                                     
2003-05-05  340.35  342.59  339.73  341.31   11948     Monday      125   
2003-05-06  341.53  344.21  340.28  343.28   12161    Tuesday      126   
2003-05-07  343.45  344.19  339.00  341.23   11588  Wednesday      127   
2003-05-08  341.28  348.13  339.96  347.38   12108   Thursday      128   
2003-05-09  347.33  348.58  345.82  347.92   11539     Friday      129   

            Daily_return  Log_daily_Return  
Date                                        
2003-05-05           NaN               NaN  
2003-05-06      0.577188          0.005755  
2003-05-07     -0.597180         -0.005990  
2003-05-08      1.802303          0.017863  
2003-05-09      0.155449          0.001553  
            Open_XAG  High_XAG  Low_XAG  Close_XAG  Volume_XAG Weekday_XAG  \
Date                                                                         
2003-08-08

In [4]:
#Merge the 3 datasets
data_temp1 = pd.concat([data_xau, data_xag, data_eur], axis=1)
print(data_temp1.head())

              Open    High     Low   Close   Volume    Weekday  Yearday  \
Date                                                                      
2003-05-05  340.35  342.59  339.73  341.31  11948.0     Monday    125.0   
2003-05-06  341.53  344.21  340.28  343.28  12161.0    Tuesday    126.0   
2003-05-07  343.45  344.19  339.00  341.23  11588.0  Wednesday    127.0   
2003-05-08  341.28  348.13  339.96  347.38  12108.0   Thursday    128.0   
2003-05-09  347.33  348.58  345.82  347.92  11539.0     Friday    129.0   

            Daily_return  Log_daily_Return  Open_XAG  ...  \
Date                                                  ...   
2003-05-05           NaN               NaN       NaN  ...   
2003-05-06      0.577188          0.005755       NaN  ...   
2003-05-07     -0.597180         -0.005990       NaN  ...   
2003-05-08      1.802303          0.017863       NaN  ...   
2003-05-09      0.155449          0.001553       NaN  ...   

            Log_daily_Return_XAG  Open_EUR  Hi

In [5]:
print(data_temp1.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Weekday', 'Yearday',
       'Daily_return', 'Log_daily_Return', 'Open_XAG', 'High_XAG', 'Low_XAG',
       'Close_XAG', 'Volume_XAG', 'Weekday_XAG', 'Yearday_XAG',
       'Daily_return_XAG', 'Log_daily_Return_XAG', 'Open_EUR', 'High_EUR',
       'Low_EUR', 'Close_EUR', 'Volume_EUR', 'Weekday_EUR', 'Yearday_EUR',
       'Daily_return_EUR', 'Log_daily_Return_EUR'],
      dtype='object')


In [6]:
#Drop repeat columns
data_temp1.drop(columns=['Weekday_XAG','Yearday_XAG','Weekday_EUR','Yearday_EUR'], inplace=True)
print(data_temp1.head())

              Open    High     Low   Close   Volume    Weekday  Yearday  \
Date                                                                      
2003-05-05  340.35  342.59  339.73  341.31  11948.0     Monday    125.0   
2003-05-06  341.53  344.21  340.28  343.28  12161.0    Tuesday    126.0   
2003-05-07  343.45  344.19  339.00  341.23  11588.0  Wednesday    127.0   
2003-05-08  341.28  348.13  339.96  347.38  12108.0   Thursday    128.0   
2003-05-09  347.33  348.58  345.82  347.92  11539.0     Friday    129.0   

            Daily_return  Log_daily_Return  Open_XAG  ...  Volume_XAG  \
Date                                                  ...               
2003-05-05           NaN               NaN       NaN  ...         NaN   
2003-05-06      0.577188          0.005755       NaN  ...         NaN   
2003-05-07     -0.597180         -0.005990       NaN  ...         NaN   
2003-05-08      1.802303          0.017863       NaN  ...         NaN   
2003-05-09      0.155449          0.

In [7]:
#Remove empty rows from Silver Data
data_temp2 = data_temp1[data_temp1.index>='2003-08-08']

In [8]:
print(data_temp2.head())

              Open    High     Low   Close   Volume    Weekday  Yearday  \
Date                                                                      
2003-08-08  352.30  357.19  351.68  356.30  17176.0     Friday    220.0   
2003-08-10  355.80  355.80  355.21  355.27    352.0     Sunday    222.0   
2003-08-11  355.80  361.78  355.17  361.57  18463.0     Monday    223.0   
2003-08-12  361.60  361.71  357.36  358.48  20332.0    Tuesday    224.0   
2003-08-13  357.92  363.60  354.95  361.35  20764.0  Wednesday    225.0   

            Daily_return  Log_daily_Return  Open_XAG  ...  Volume_XAG  \
Date                                                  ...               
2003-08-08      1.184222          0.011773     4.966  ...      6015.0   
2003-08-10     -0.289082         -0.002895     4.925  ...        33.0   
2003-08-11      1.773299          0.017578     4.915  ...      2142.0   
2003-08-12     -0.854606         -0.008583     4.925  ...      2281.0   
2003-08-13      0.800603          0.

In [9]:
#Fill na values (forward fill)
data = data_temp2.fillna(method='ffill')

In [10]:
print(data.head())

              Open    High     Low   Close   Volume    Weekday  Yearday  \
Date                                                                      
2003-08-08  352.30  357.19  351.68  356.30  17176.0     Friday    220.0   
2003-08-10  355.80  355.80  355.21  355.27    352.0     Sunday    222.0   
2003-08-11  355.80  361.78  355.17  361.57  18463.0     Monday    223.0   
2003-08-12  361.60  361.71  357.36  358.48  20332.0    Tuesday    224.0   
2003-08-13  357.92  363.60  354.95  361.35  20764.0  Wednesday    225.0   

            Daily_return  Log_daily_Return  Open_XAG  ...  Volume_XAG  \
Date                                                  ...               
2003-08-08      1.184222          0.011773     4.966  ...      6015.0   
2003-08-10     -0.289082         -0.002895     4.925  ...        33.0   
2003-08-11      1.773299          0.017578     4.915  ...      2142.0   
2003-08-12     -0.854606         -0.008583     4.925  ...      2281.0   
2003-08-13      0.800603          0.

In [11]:
#Save data for analysis
data.to_csv('data/Analysis/D1_XAUXAGEUR_Returns.csv')